## Creating a K8S Cluster on an M1 Mac

At this point in time, Macs with the M1 line of processors work best with *k3d*, which installs k3s inside of containers runnign in Docker.  

We start by ensuring we have not already created our cluster:

Install Docker:

https://docs.docker.com/desktop/mac/install/
(Select Mac with Apple Chip)

Follow the installation instructions to install Docker Desktop for your OS.

After Docker destkop has been installed, launch docker desktop then proceed with installing k3d:


In [1]:
brew install k3d

==> Downloading https://ghcr.io/v2/homebrew/core/k3d/manifests/5.2.2
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/k3d/blobs/sha256:94ab53847ee937
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Pouring k3d--5.2.2.arm64_big_sur.bottle.tar.gz
==> Caveats
zsh completions have been installed to:
  /opt/homebrew/share/zsh/site-functions
==> Summary
🍺  /opt/homebrew/Cellar/k3d/5.2.2: 9 files, 18.9MB
==> Running `brew cleanup k3d`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).


Ensure k3d was installed correctly with:

In [8]:
brew list | grep -i k3d

k3d


In [2]:
k3d cluster list

NAME   SERVERS   AGENTS   LOADBALANCER


<br>

The following command creates a cluster named *test*.  It then maps localhost ports 8081 and 8443 to the cluster's LoadBalancer service on ports 80 and 443, where Traefik is running as an ingress controller.  It creates a cluster with 1 control-plane node and 2 worker/agent nodes, exposing the control plane using an API server running on port 6550:

In [4]:
k3d cluster create --api-port 6550 -p "8081:80@loadbalancer" -p "8443:443@loadbalancer" --agents 2 test

INFO[0000] portmapping '8443:443' targets the loadbalancer: defaulting to [servers:*:proxy agents:*:proxy] 
INFO[0000] portmapping '8081:80' targets the loadbalancer: defaulting to [servers:*:proxy agents:*:proxy] 
INFO[0000] Prep: Network                                
INFO[0000] Created network 'k3d-test'                   
INFO[0000] Created volume 'k3d-test-images'             
INFO[0000] Starting new tools node...                   
INFO[0000] Starting Node 'k3d-test-tools'               
INFO[0001] Creating node 'k3d-test-server-0'            
INFO[0001] Creating node 'k3d-test-agent-0'             
INFO[0001] Creating node 'k3d-test-agent-1'             
INFO[0001] Creating LoadBalancer 'k3d-test-serverlb'    
INFO[0001] Using the k3d-tools node to gather environment information 
INFO[0002] Starting cluster 'test'                      
INFO[0002] Starting servers...                          
INFO[0002] Starting Node 'k3d-test-server-0'            
INFO[0006] Starting agents... 

In [5]:
kubectl get nodes

NAME                STATUS   ROLES                  AGE   VERSION
k3d-test-agent-1    Ready    <none>                 22s   v1.21.7+k3s1
k3d-test-agent-0    Ready    <none>                 22s   v1.21.7+k3s1
k3d-test-server-0   Ready    control-plane,master   31s   v1.21.7+k3s1


In [6]:
kubectl get pods -A

NAMESPACE     NAME                                      READY   STATUS              RESTARTS   AGE
kube-system   local-path-provisioner-5ff76fc89d-rdhhg   1/1     Running             0          30s
kube-system   metrics-server-86cbb8457f-f9sbq           1/1     Running             0          30s
kube-system   coredns-7448499f4d-9fwq4                  1/1     Running             0          30s
kube-system   helm-install-traefik-crd-98st8            0/1     Completed           0          31s
kube-system   traefik-6b84f7cbc-mznc4                   0/1     ContainerCreating   0          3s
kube-system   svclb-traefik-mm4qb                       0/2     ContainerCreating   0          3s
kube-system   svclb-traefik-v84tv                       0/2     ContainerCreating   0          3s
kube-system   svclb-traefik-kxl6r                       0/2     ContainerCreating   0          3s
kube-system   helm-install-traefik-cs2bw                0/1     Completed           2          31s


<br>

## dnsmasq

From this point, we'd like to have DNS resolve certain DNS names to localhost for our cluster.  For this, we're going to use *dnsmasq*, which allows us to override wildcard DNS entries and point them to localhost.  Since we're on a Mac, we're going to use 'brew' to install it.  In this case, we've already installed it, so no changes are made:

In [7]:
brew install dnsmasq

==> Downloading https://ghcr.io/v2/homebrew/core/dnsmasq/manifests/2.86
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/dnsmasq/blobs/sha256:958b73b470
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Pouring dnsmasq--2.86.arm64_big_sur.bottle.tar.gz
==> Caveats
To restart dnsmasq after an upgrade:
  sudo brew services restart dnsmasq
Or, if you don't want/need a background service you can just run:
  /opt/homebrew/opt/dnsmasq/sbin/dnsmasq --keep-in-foreground -C /opt/homebrew/etc/dnsmasq.conf -7 /opt/homebrew/etc/dnsmasq.d,*.conf
==> Summary
🍺  /opt/homebrew/Cellar/dnsmasq/2.86: 10 files, 614.9KB
==> Running `brew cleanup dnsmasq`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).


In [9]:
brew list | grep -i dnsmasq

dnsmasq


In your Jupyter notebook working directory, create a file called "password" and put your root password in this file. This is to enable Jupyter notebook to execute sudo commands without exposing your root password or storing it in your machines CLI history while creating a variable. 

<br>

Next we will add a rule to the configuration for dnsmasq that routes any URL ending in *.test* to 127.0.0.1.  The code below checks first to see if that line already exists and adds it if it does not:

In [14]:
if grep -Fxq "address=/.test/127.0.0.1" /opt/homebrew/etc/dnsmasq.conf
then
    echo "DNSMasq rule for .test top-level domain already exists"
else
    cat password | sudo -S echo "address=/.test/127.0.0.1" >> /opt/homebrew/etc/dnsmasq.conf
    grep "address=/.test/127.0.0.1" /opt/homebrew/etc/dnsmasq.conf
fi

DNSMasq rule for .test top-level domain already exists


<br>

From here we will need to use *brew* to restart the dnsmasq service so that the new configuration can be loaded & used:

In [15]:
cat password | sudo -S brew services restart dnsmasq

==> Tapping homebrew/services
Cloning into '/opt/homebrew/Library/Taps/homebrew/homebrew-services'...
remote: Enumerating objects: 1656, done.
remote: Counting objects: 100% (535/535), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 1656 (delta 229), reused 356 (delta 130), pack-reused 1121
Receiving objects: 100% (1656/1656), 481.02 KiB | 1.57 MiB/s, done.
Resolving deltas: 100% (705/705), done.
Tapped 1 command (44 files, 616.0KB).
  /opt/homebrew/Cellar/dnsmasq/2.86/sbin
  /opt/homebrew/Cellar/dnsmasq/2.86/sbin/dnsmasq
  /opt/homebrew/opt/dnsmasq
  /opt/homebrew/opt/dnsmasq/sbin
  /opt/homebrew/var/homebrew/linked/dnsmasq
This will require manual removal of these paths using `sudo rm` on
brew upgrade/reinstall/uninstall.
==> Successfully started `dnsmasq` (label: homebrew.mxcl.dnsmasq)


<br>

Use `dig` to test that URLs ending in `.test` resolve to 127.0.0.1 on the DNS server running at 127.0.0.1.  The `@127.0.0.1` option tells dig to use the DNS server running on localhost, the `dnsmasq` service we started earlier.  Note that the `ANSWER SECTION` provides the response for `fubar.test` as `127.0.0.1`.

In [16]:
dig @127.0.0.1 fubar.test


; <<>> DiG 9.10.6 <<>> @127.0.0.1 fubar.test
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 8691
;; flags: qr aa rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;fubar.test.			IN	A

;; ANSWER SECTION:
fubar.test.		0	IN	A	127.0.0.1

;; Query time: 0 msec
;; SERVER: 127.0.0.1#53(127.0.0.1)
;; WHEN: Sat Jan 08 13:14:41 MST 2022
;; MSG SIZE  rcvd: 55



<br>

Next, we'll want to override the Mac's DNS resolver for all URLs ending in `.test`, sending only those URLs to our dnsmasq service.  The mac uses "/etc/resolver" for this purpose.  This will allow us to use any browser, such as Chrome or Safari, to use `dnsmasq` to resolve URLs ending in `.test`.

If there is a file in `/etc/resolver` with a filename that matches the end of the URL, then the contents of that file will identify the DNS server to use for that URL.  In our case, we create a file named `/etc/resolver/test` that configures handling of all URLs ending in `.test`, a fictitious top-level domain that I created for test purposes.  This easily could be a subdomain of an actual domain owned by the cluster operator (for example, to listen to any URLs ending in `.k8s.example.com`, the full filename would be `/etc/resolver/k8s.example.com`

In [18]:
cat password | sudo -S mkdir -p /etc/resolver

In [21]:
if grep -Fxq "nameserver 127.0.0.1" /etc/resolver/test
then
    echo "MacOS DNS resolver for .test top-level domain already defers to localhost"
else
    cat password | sudo -S sh -c 'echo "nameserver 127.0.0.1" >> /etc/resolver/test'
    cat /etc/resolver/test
fi

grep: /etc/resolver/test: No such file or directory
Password:nameserver 127.0.0.1


<br>

### Testing
We will test that the entire process works from end-to-end by deploying an NGINX pod to our local k8s cluster, exposing it using an Ingress resource, and then access the pod in our web browser:

In [ ]:
brew install kubectl

In [22]:
kubectl create deployment nginx --image=nginx

deployment.apps/nginx created


In [23]:
kubectl create service clusterip nginx --tcp=80:80

service/nginx created


In [24]:
cat << EOF > ingress.yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: nginx
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
spec:
  rules:
  - host: nginx.test
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: nginx
            port:
              number: 80
EOF

In [25]:
kubectl apply -f ingress.yaml

ingress.networking.k8s.io/nginx created


<br>
Recall that when we started our k3d cluster, we mapped the cluster's ingress controller service's port 80 & 443 to localhost ports 8081 and 8443.  We also configured 'dnsmasq' to resolve all URLs ending in '.test' to 127.0.0.1.  This allows us to use our web browser to visit HTTP & HTTPS services exposed using ingress objects inside our local k3d cluster.

Open a browser window to [http://nginx.test:8081](http://nginx.test:8081) and confirm that you can see the NGINX starter page.  Likewise, you will be able to visit https://nginx.test:8443 (after accepting TLS certificate errors) and also see the same page.  Likewise, confirm that http://bad-url.test:8081 also resolves to the ingress controller, but gives a `404 page not found` error.
<br>
### Cleanup
After confirming that the cluster works as expected, let's remove our test instance of nginx:

In [2]:
kubectl delete ingress nginx

ingress.networking.k8s.io "nginx" deleted


In [3]:
kubectl delete service nginx

service "nginx" deleted


In [4]:
kubectl delete deployment nginx

deployment.apps "nginx" deleted


<br>
Our local cluster is now ready for use.  This cluster can be used for all of our Nephtek workbooks.